In [19]:
#import all required packages

import numpy as np
from time import time, sleep
import random
from IPython.display import clear_output
import pickle

In [20]:
# functions for displaying board and checking for wins

def drawBoard(board): 
    print( " ",  board[6], " | ", board[7], " | ", board[8])
    print("-----------------")
    print( " ", board[3], " | ", board[4], " | ", board[5])
    print("-----------------")
    print( " ", board[0], " | ", board[1], " | ", board[2])
        
def checkVertical(positions):
    if(  (positions[6]==positions[3]==positions[0]==2)
      or (positions[7]==positions[4]==positions[1]==2)
      or (positions[8]==positions[5]==positions[2]==2)):
        return 2
    if(  (positions[6]==positions[3]==positions[0]==1)
      or (positions[7]==positions[4]==positions[1]==1)
      or (positions[8]==positions[5]==positions[2]==1)):
        return 1
    return 0

def checkHorizontal(positions):
    if(  (positions[6]==positions[7]==positions[8]==2)
      or (positions[3]==positions[4]==positions[5]==2)
      or (positions[0]==positions[1]==positions[2]==2)):
        return 2
    if(  (positions[6]==positions[7]==positions[8]==1)
      or (positions[3]==positions[4]==positions[5]==1)
      or (positions[0]==positions[1]==positions[2]==1)):
        return 1
    return 0
    
def checkDiagonal(positions):
    if(  (positions[6]==positions[4]==positions[2]==2)
      or (positions[0]==positions[4]==positions[8]==2)):
        return 2
    if(  (positions[6]==positions[4]==positions[2]==1)
      or (positions[0]==positions[4]==positions[8]==1)):
        return 1
    return 0
    
def checkAnyWin(board):
    if(checkVertical(board)==1 or checkHorizontal(board)==1 or checkDiagonal(board)==1 ):
        return 1
    if(checkVertical(board)==2 or checkHorizontal(board)==2 or checkDiagonal(board)==2 ):
        return 2
    return 0    

In [21]:
# Q tables for each player 

n_states = 3**9
n_actions = 9
Q1 = np.zeros((n_states, n_actions))  # Q-table for Q-learning
Q2 = np.zeros((n_states, n_actions))  # Q-table for Q-learning

In [22]:
#Q states generator 

Q=[]
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                for m in range(3):                    
                    for n in range(3):
                        for o in range(3):
                            for p in range(3):
                                for q in range(3):
                                    Q.append([i,j,k,l,m,n,o,p,q])
print(len(Q))

19683


In [32]:
# functions for learning Q values

lr = 0.001
df = 0.9
    
#this function will check for possible future wins for both, opponent and itself.
#accordingly it will either block opponent win or choose winning move 

def win_checker(board, remain, player):       
    b = board
    if(player):
        for i in remain:
            b[i]=1
            x = checkAnyWin(b)
            b[i]=0            
            if(x==1):
                return i
    else:
        for i in remain:
            b[i]=2
            x = checkAnyWin(b)
            b[i]=0            
            if(x==2):
                return i
    return -1
            
#choose action according to epsilon greedy function
def choose_action(board, remain, player):
    a = "win"
    b = "block"
    c = "random"
    d = "nothing"
    y = win_checker(board,remain, player )  # checking own win
    if(y!=-1):
        return y, a
    x = win_checker(board,remain, (not player) ) # blocking opponent win
    if(x!=-1):
        return x, b
    else:
        if random.uniform(0, 1) < epsilon:
            return random.choice(remain), c
        else:
            if(player):
                maxx = remain[0]
                b = Q.index(board)
                for i in remain:
                    if(Q1[b][maxx]<Q1[b][i]):
                        maxx=i
                return maxx, d
            else:
                maxx = remain[0]
                b = Q.index(board)
                for i in remain:
                    if(Q2[b][maxx]<Q2[b][i]):
                        maxx=i
                return maxx, d
    
# update q tables for players 
    
def update_q_table(state, action, reward, next_state, player):
    if(player):
        best_next_action = np.argmax(Q1[Q.index(next_state)])
        td_target = reward + df * Q1[Q.index(next_state)][best_next_action]
        td_error = td_target - Q1[Q.index(state)][action]
        Q1[Q.index(state)][action] += lr * td_error
    else:
        best_next_action = np.argmax(Q2[Q.index(next_state)])
        td_target = reward + df * Q2[Q.index(next_state)][best_next_action]
        td_error = td_target - Q2[Q.index(state)][action]
        Q2[Q.index(state)][action] += lr * td_error


In [ ]:
# Training and game running code
# each player will play first alternatively
# rewards:
# 30 for win
# 10 for tie
# 1 for playing on a valid position 
# -15 for playing on already played position

epsilon = 0.1

one, two, tie=0, 0, 0
error = 0
n_episodes = 10000
for episode in range(n_episodes):
    board = [0,0,0,0,0,0,0,0,0]
    remain= [0,1,2,3,4,5,6,7,8]
    value=episode%2
    x=0
    for i in range(9):
        next_board=board
        if(i%2==value):
            if(i<0):
                action, win = random.choice(remain)
            else:
                action, win = choose_action(board, remain, True)
            if(board[action]==0):
                next_board[action] = 1
                remain.remove(action)
                update_q_table(board, action, 1, next_board, True)
                if(win=="win" or win=="block"):
                    update_q_table(board, action, 20, next_board, True)                    
            else:
                update_q_table(board, action, -15, next_board, True)
                error+=1
                x=-1
                break
        
        else:
            if(i<0):
                action, win = random.choice(remain)
            else:
                action, win = choose_action(board, remain, False)
            if(board[action]==0):
                next_board[action] = 2
                remain.remove(action)
                update_q_table(board, action, 5, next_board, False)
                if(win=="win" or win=="block"):
                    update_q_table(board, action, 20, next_board, False)       
            else:
                update_q_table(board, action, -15, next_board, False)
                error+=1
                x=-1
                break

        x = (checkAnyWin(next_board))
        if(x==1):
            one+=1
            update_q_table(board, action, 30, next_board, True)
            update_q_table(board, action, -15, next_board, False)
            break
        elif(x==2):
            two+=1
            update_q_table(board, action, -15, next_board, True)
            update_q_table(board, action, 30, next_board, False)
            break
            
        board = next_board
    if(x==0):
        tie+=1
        update_q_table(board, action, 10, next_board, False)
        update_q_table(board, action, 10, next_board, True)
    epsilon*=0.99
        

    if(episode%200==0):
        print(episode)

# one = wins by player 1
# two = wins by player 2
print(one, two, tie, error)
drawBoard(board)

In [45]:
#watch it play against each other

epsilon = 0.1

board = [0,0,0,0,0,0,0,0,0]
remain= [0,1,2,3,4,5,6,7,8]
for i in range(9):
    if(i%2==1):
        move, win = choose_action(board, remain, True) # 1st bool = True = agent 1 false = agent 2
        print("Agent tried to play on position: ", move)
        
        if(board[move]==0):
            board[move]=1
            remain.remove(move)
        else:
            print("Agent lose")
            break

    else:
        move, win = choose_action(board, remain, False)
        print("Agent 2 tried to play on position: ", move)

        if(board[move]==0):
            board[move]=2
            remain.remove(move)
        else:
            print("Agent 2 lose")
            break

    clear_output() 
    drawBoard(board)  
    x = checkAnyWin(board)
    if(x!=0):
        if(x==1):
            print("Agent 1 won")
        elif(x==2):
            print("Agent 2 won")
        break
    sleep(1) 
    clear_output()

  1  |  0  |  2
-----------------
  1  |  2  |  0
-----------------
  2  |  1  |  2
Agent 2 won


In [2]:
import pickle

In [42]:
Q1 = np.zeros((n_states, n_actions))  # Q-table for Q-learning
Q2 = np.zeros((n_states, n_actions))  # Q-table for Q-learning

In [ ]:
# store 

with open('q1.xyz', 'wb') as fp:
    pickle.dump(Q1, fp)
with open('q2.xyz', 'wb') as fp:
    pickle.dump(Q2, fp)

In [46]:
with open('q1.xyz', 'rb') as fp:
    Q1 = pickle.load(fp)
with open('q2.xyz', 'rb') as fp:
    Q2 = pickle.load(fp)

In [47]:
# play against agent 1

board = [0,0,0,0,0,0,0,0,0]
remain= [0,1,2,3,4,5,6,7,8]
first = 0 # if 0 agent will play first else player will play first

for i in range(9):
    if(i%2==first): 
        move,win = choose_action(board, remain, True)
        print("Agent tried to play on position: ", move)
        
        if(board[move]==0):
            board[move]=1
            remain.remove(move)
        else:
            print("Agent lose")
            break
            
    else:
        a = int(input()) 
        print("Player tried to play on position: ", move)

        if(board[a-1]==0):
            board[a-1]=2
            remain.remove(a-1)
        else:
            print("Player lose")
            break 

    clear_output() 
    drawBoard(board)  
    x = checkAnyWin(board)
    if(x!=0):
        if(x==1):
            print("Agent 1 won")
        elif(x==2):
            print("Agent 2 won")
        break
    sleep(1) 

  1  |  2  |  1
-----------------
  2  |  2  |  1
-----------------
  1  |  1  |  2


In [49]:
# play against agent 2

board = [0,0,0,0,0,0,0,0,0]
remain= [0,1,2,3,4,5,6,7,8]

# if 0 agent will play first else player will play first
first = 0 

for i in range(9):
    if(i%2==first): 
        move,win = choose_action(board, remain, False)
        print("Agent tried to play on position: ", move)
        
        if(board[move]==0):
            board[move]=2
            remain.remove(move)
        else:
            print("Agent lose")
            break
            
    else:
        a = int(input()) 
        print("Player tried to play on position: ", move)

        if(board[a-1]==0):
            board[a-1]=1
            remain.remove(a-1)
        else:
            print("Player lose")
            break 

    clear_output() 
    drawBoard(board)  
    x = checkAnyWin(board)
    if(x!=0):
        if(x==1):
            print("Agent 1 won")
        elif(x==2):
            print("Agent 2 won")
        break
    sleep(1) 

  2  |  2  |  1
-----------------
  1  |  1  |  2
-----------------
  2  |  2  |  1
